Алгебраический многочлен

In [462]:
import numpy as np
import pandas as ps
from scipy import linalg
x = np.array([0.87267, 1.22173, 1.57080, 1.91986, 2.26893, 2.61799, 2.96706])
f = np.array([0.00123, 0.01343, 0.08411, 0.37341, 1.31146, 3.88447, 10.10742])

In [463]:
#возвращает коэффициенты алгебраического многочлена
def alg_coef(x,f):
    length = len(x)
    
#поиск столбца коэффициентов а[i] многочлена
    array = np.zeros((length,length))
    for i in range(0,length):
        for j in range(0,length):
            array[i][j] = x[i]**j
    a = linalg.solve(array,f)
    return a

#таблица коэффициентов алгебраического многочлена(надо редактировать при смене размерности x,f)
table1 = ps.DataFrame(data=alg_coef(x,f).reshape(1,length), index=np.arange(0,1), columns=np.arange(0,length))
table1.columns = ['a0','a1','a2','a3','a4','a5','a6']
table1.index = [' ']
table1

,a0,a1,a2,a3,a4,a5,a6
,0.983067,-4.142168,7.193307,-6.658516,3.519626,-1.036111,0.142891


In [425]:
#возвращает значения алгебраического полинома в х0
def alg_pol(x,f,x0):
    length = len(x)
    
#матрица степеней х
    xx = np.zeros(length)
    for i in range(0,length):
        xx[i] = x0**i

    return alg_coef(x,f).dot(xx)

In [426]:
alg_pol(x,f,1.22173)

0.013429999999999609

Сплайн

In [428]:
#подсчет производной алг. полинома в точке
def derivative_alg_pol(x,f,x0):
    length = len(x)

#матрица степеней х производной
    der = np.zeros(length)
    for i in range(1,length):
        der[i] = i * x0**(i-1)
    
    return alg_coef(x,f).dot(der)

In [450]:
#возвращает коэффициенты i-го сплайна
def spline_coef(i,x,f):
    d = derivative_alg_pol(x,f,x[i])
    dd = derivative_alg_pol(x,f,x[i+1])
    
    a_s = np.zeros(4)
    a_s[0] = (-dd * x[i]**2 * x[i+1] * (x[i+1] - x[i]) + f[i+1] * x[i]**2 * (3*x[i+1] - x[i]))/(x[i+1] - x[i])**3  +  (-d * x[i+1]**2 * x[i] * (x[i+1] - x[i]) + f[i] * x[i+1]**2 * (x[i+1] - 3 * x[i]))/(x[i+1] - x[i])**3
    a_s[1] = (dd * x[i] * (2 * x[i+1] + x[i]) * (x[i+1] - x[i]) - 6 * (f[i+1] - f[i]) * x[i] * x[i+1])/(x[i+1] - x[i])**3  +  (d * x[i+1] * (x[i+1] + 2 * x[i]) * (x[i+1] - x[i]))/(x[i+1] - x[i])**3
    a_s[2] = (-dd * (2 *x[i] + x[i+1]) * (x[i+1] - x[i]) + 3 * (f[i+1] - f[i]) * (x[i] + x[i+1]))/(x[i+1] - x[i])**3  -  (d * (x[i] + 2 * x[i+1]) * (x[i+1] - x[i]))/(x[i+1] - x[i])**3
    a_s[3] = (dd * (x[i+1] - x[i]) - 2 * (f[i+1] - f[i]) + d * (x[i+1] - x[i]))/(x[i+1] - x[i])**3
    return a_s

#таблица коэффициентов i-го сплайна
spline_coef_tab = np.zeros((6,4))
for i in range(0,len(f)-1):
    spline_coef_tab[i] = spline_coef(i,x,f).reshape(1,4)
table = ps.DataFrame(data=spline_coef_tab, index=np.arange(0,6), columns=np.arange(0,4))
table.columns = ['a0','a1','a2','a3']
table.index = ['S1','S2','S3','S4','S5','S6']
table

,a0,a1,a2,a3
S1,0.124560,-0.271844,0.153334,-0.004321
S2,-0.959060,2.386662,-2.020752,0.588323
S3,-4.906811,9.843294,-6.714942,1.573246
S4,-20.195637,33.534605,-18.951298,3.679752
S5,-67.216156,95.354669,-46.042996,7.637131
S6,-185.931363,230.854166,-97.594448,14.174695


In [451]:
#значение i-го кубического сплайна S3i в х0
def spline(i,x,f,x0):
    
#матрица степеней х сплайна 3 степени    
    xx = np.zeros(4)
    for j in range(0,4):
        xx[j] = x0**j
        
    return spline_coef(i,x,f).dot(xx)

In [452]:
#выдает значение функции в точке - интерполяция
def spline_pol(x,f,x0):
    length = len(x)
    answer = 0
    parametr = 0
        
    while parametr < length:
        
        if (x[parametr] <= x0) & (x0 <= x[parametr+1]) :
            answer = spline(parametr,x,f,x0)
            parametr = 11
        else:
            parametr += 1
            
    return answer

In [456]:
def qwe(x0):
    tab = np.array([alg_pol(x,f,x0),spline_pol(x,f,x0)])
    table = ps.DataFrame(data=tab.reshape(1,2), index=np.arange(0,1), columns=np.arange(0,2))
    table.columns = ['alg', 'spline']
    table.index = [' ']
    return table

In [457]:
qwe(1.65)

,alg,spline
,0.120859,0.120415


In [403]:
qwer = np.zeros((2,7))
for i in range(0,7):
    qwer[0][i] = alg_poll(x,f,x[i])
    qwer[1][i] = spline_interpolate(x,f,x[i])
table = ps.DataFrame(data=qwer, index=np.arange(0,2), columns=np.arange(0,7))
table

,0,1,2,3,4,5,6
0,0.00123,0.01343,0.08411,0.37341,1.31146,3.88447,10.10742
1,0.00123,0.01343,0.08411,0.37341,1.31146,3.88447,10.10742


In [312]:
def spline_interp0late(x,f,x0):
    length = len(x)
    S = np.zeros(length - 1)
    answer = 0
    v = 0
    
    for i in range(0,length - 1):
        S[i] = spline(i,x,f,x0)
        
    while v < length:
        if (x[v] <= x0) & (x0 < x[v+1]) :
            w = S[v]
            v=11
        else:
            v+=1
    return w

spline_interp0late(x,f,2.26893)

1.3114599999998404